In [1]:
'''Connecting to DB'''
import pyodbc
Server='qa-datagov01.nexjqa.local'
DB='PRO_MAJIK_CARMAX'
Ver='sql2012'
user='nexj'
password='nexj'

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+Server+';DATABASE='+DB+';UID='+user+';PWD='+password)
# Create a cursor from the connection
cursor = cnxn.cursor()

In [2]:
# '''getting the list of equipments'''
# sqlStr='SELECT [id]  FROM [PRO_MAJIK_CARMAX].[dbo].[equipment]'  
# cursor.execute(sqlStr)    
# buffers = cursor.fetchall()
# equipments=[]
# # eqStr="("

# for buffer in buffers: 
#     equipments.append (buffer[0]) 
# #print (equipments)
equipments=[]
equipments.append ("equipment")

In [3]:
class Device(object):
    """Each  of the factury line:
    Assumtion:
        We are not sending the values that are zero or negative when creating the class(we filter them out)
    Attributes:
        id: A string representing the Device id  .
        errors: dictinary of the errors,   id as the key  and their MTTF and MeanTimeDown as values [ErrorType,MTTF,MeanTimeDown]
        Line number: the current line 
        Run Time: the run time of the device  , dictianry with error id as the key , MTTF- running time of the device since the last maintance for that error as value if it gets close to a threshold we notify the system admion 
    """
    
    def __init__(self, deviceId,Line):
        "Initilize the values"
        self.id = deviceId
        self.line=Line
        self.errors={}#dict()
        self.runTime={}
        self.maintenanceFlag=False  # this should be array for each error but for now just use one flag 
        
    def get_deviceId(self):
        return self.id
    def get_line(self):
        return self.line

    def get_runTime(self):
        return self.runTime
    
    def update_deviceId(self, newDeviceId):
        self.id=newDeviceId
    def update_line(self,newLine):
        self.line=newLine
    def get_error_size(self):
        '''Return the number of errors that are associated with each device '''
        return len(self.runTime)
    
    
    def add_runtimes(self,deviceId,errorID,MTTF,ErrorType):
        "Assign the values of runtime associated with the device"
        if(self.id!=deviceId):
            print("WRONG DEVICE!")
            return
        self.runTime[errorID]=MTTF

        
    def update_RunTime(self, hours):
        "updating the run time after some amopunt of running time of the device"
        for errorID in self.runTime:
            self.runTime[errorID]-=hours
            
    def reset_RunTime(self,deviceId,errorID,MTTF,ErrorType):
        "if a fix for an error id happens for a device the runtime value will be updated to the original value "
        if(errorID  in self.runTime):
            self.maintenanceFlag=False  # if it was the array then we use the error to index it 
            self.runTime[errorID]=MTTF
               
    def get_maintenanceFlag(self):
        return self.maintenanceFlag
            
    def perdicted_error(self, threshold):
        "Return a list of errors for the device that can happen in the next given threshold time"
        maintenance=[]
        for errorID in self.runTime:
            if self.runTime[errorID] <= threshold:
                    self.maintenanceFlag=True
                    maintenance.append(errorID)
        return maintenance
    
    def print_device(self):
        print (self.runTime)
            
    

In [4]:
eqi=[]
#DEFINEING THE ROW NAMEs
line=0
ErrorId=1
ErrorType=2
DeviceId=3
TFsum=4
DownTimeSum=5
Fnum=6

 #if FLAG is 1 add runtime from error if 0 add from runtable
FLAG=1
# FLAG=0

for equipment in equipments:        
        sqlStr=' select * FROM [PRO_MAJIK_CARMAX].[dbo].[MTTFtbl]  WHERE LTRIM(RTRIM([DeviceId]))=LTRIM(RTRIM(\''+equipment+'\')) AND  [TFsum] > 0 ' 
#         print (sqlStr)
        cursor.execute(sqlStr)    
        '''Read all at once '''
        rows = cursor.fetchall()
        buffer=Device(equipment,1)
        for row in rows:
            if(row[Fnum]==0):
                MTTF=0
                MeanDownT=0
            else:
                MTTF=row[TFsum]/row[Fnum]
#                 MeanDownT=row[DownTimeSum]/row[Fnum]
            if(FLAG==1):
                buffer.add_runtimes(equipment,row[ErrorId],MTTF,row[ErrorType])

        
        if(FLAG==0):
            '''Fetching from run table'''
            sqlStr=' select * FROM [PRO_MAJIK_CARMAX].[dbo].[RUNtbl]  WHERE LTRIM(RTRIM([DeviceId]))=LTRIM(RTRIM(\''+equipment+'\')) AND  [MTTF] > 0 ' 
            cursor.execute(sqlStr)    
            '''Read all at once '''
            run_rows = cursor.fetchall()
  
            for row in run_rows:  
                MTTF=row[TFsum]
                buffer.add_runtimes(equipment,row[ErrorId],MTTF,row[ErrorType])
        
            
           
        eqi.append(buffer)

In [5]:
for e in eqi:
    print((e.get_deviceId(),e.get_error_size()))

('equipment', 124)


In [10]:
eqStr="([ErrorId],[line],[ErrorType],[DeviceId],[MTTF])" 
line=1
for e in eqi:
    # update the time for errors (let's say that we have been running it for 2 hours)
    e.update_RunTime(2000)
    device_name=e.get_deviceId()  #FOR NOW WE DON'T NEED THIS 
    #ERROR TYPE FOR NOW IS JUST A STRING 
    runList= e.get_runTime()
    
    for key in runList:
        strWrite=""
        strWrite+="\'"+key+"\'"+','
        strWrite+=str(line)+','
        strWrite+="\'"+"UNPLANNED"+"\'"+',' #for now error type and id are the same
        strWrite+="\'"+"equipment"+"\'"+','#we don't have a name for it yet
        strWrite+=str(runList[key])
    
        #writing to DB 
        sqlStr=' UPDATE [PRO_MAJIK_CARMAX].[dbo].[RUNtbl] set [MTTF]='+str(runList[key])+' WHERE  [ErrorId] = \''+key+'\' AND [DeviceId]=\''+"equipment"+'\';  INSERT INTO [PRO_MAJIK_CARMAX].[dbo].[RUNtbl]'+eqStr+' select '+strWrite+' WHERE NOT EXISTS(select * From [PRO_MAJIK_CARMAX].[dbo].[RUNtbl]  WHERE  [ErrorId] = \''+key+'\'AND [DeviceId]=\''+"equipment"+'\' );'
        cursor.execute(sqlStr)   
        cnxn.commit()
    
    
    #put the threashold to 1 hour 
    needToFix=e.perdicted_error(1000)
    #if one of the errors break then we print them and fix them

    if(e.get_maintenanceFlag()==True):
        print(needToFix)
        print ("FIXING")
        for error in needToFix:
            sqlStr=' select * FROM [PRO_MAJIK_CARMAX].[dbo].[MTTFtbl]  WHERE [ErrorId]=LTRIM(RTRIM(\''+error+'\')) AND LTRIM(RTRIM([DeviceId]))=LTRIM(RTRIM(\''+device_name+'\')) AND  [TFsum] > 0 '
            cursor.execute(sqlStr)    
            '''Read all at once '''
            rows = cursor.fetchall() #this should be one item 
            for row in rows:#this should be one item 
                if(row[Fnum]==0):
                    MTTF=0
                else:
                    MTTF=row[TFsum]/row[Fnum]
            
            e.reset_RunTime(device_name,error,MTTF,ErrorType)
            
    #update the run time table !!

['A-UL', 'LC-', 'MT-Electrical', 'SU-']
FIXING
